# Homework 8
For this homework, _libsvm_ is needed. The easiest way to install this on _Mac OS X_ is using _Homebrew_ with the following command: `brew install libsvm`.

From this homework: questions 7, 8, 9 and 10 will be answered

In [1]:
from svmutil import *
import pandas as pd
from sklearn.cross_validation import KFold
from collections import Counter
import numpy as np

In [2]:
train = pd.read_table("features.train", sep=" +", header=None, engine='python')
train.columns = ["digit", "intensity", "symmetry"]
test = pd.read_table("features.test", sep=" +", header=None, engine='python')
test.columns = ["digit", "intensity", "symmetry"]

In [3]:
train.head()

,digit,intensity,symmetry
0,6,0.341092,-4.528937
1,5,0.444131,-5.496812
2,4,0.231002,-2.886750
3,7,0.200275,-3.534375
4,3,0.291936,-4.352062


In [4]:
filtered = train[(train.digit == 1) | (train.digit == 5)]
filtered["digit"].replace(1,float(-1),inplace=True)
filtered["digit"].replace(5,float(1),inplace=True)
filtered = filtered.reset_index(drop=True)

/Users/Jens/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [20]:
possible_C = np.array([0.0001, 0.001, 0.01, 0.1, 1])
runs = 100

best_run_C = np.zeros(runs)
C_val_errors = {c: np.zeros(runs) for c in possible_C}

for run in range(runs):
    kf = KFold(len(filtered), n_folds=10, shuffle=True)
    C_errors = np.zeros(len(possible_C))
    for i,C in enumerate(possible_C):
        fold_errors = np.zeros(len(kf))
        for j, index_pair in enumerate(kf):
            train_index, val_index = index_pair
            
            train_fold = filtered.ix[train_index]
            train_x = train_fold[["intensity", "symmetry"]].values.tolist()
            train_y = train_fold["digit"].values.tolist()
            
            val_fold = filtered.ix[val_index]
            val_x = val_fold[["intensity", "symmetry"]].values.tolist()
            val_y = val_fold["digit"].values.tolist()
            
            m = svm_train(train_y, train_x, '-q -t 1 -d 2 -c {} -r 1 -g 1'.format(C))
            p_label, p_acc, p_val = svm_predict(val_y, val_x, m, "-q")
            fold_errors[j] = (100-p_acc[0])/100.
            
        mean_fold_error = np.mean(fold_errors)
        C_val_errors[C][run] = mean_fold_error
        C_errors[i] = mean_fold_error
    best_run_C[run] = possible_C[np.argmin(C_errors)]

## Question 7

In [21]:
best_c = Counter(best_run_C).most_common(1)[0][0]
print("Best C: ", best_c)

Best C:  0.001


## Question 8

In [22]:
print("Cross validation error when using C=", best_c, ": ", np.mean(C_val_errors[best_c]))

Cross validation error when using C= 0.001 :  0.0047660052262


In [23]:
filtered_test = test[(test.digit == 1) | (test.digit == 5)]
filtered_test["digit"].replace(1,float(-1),inplace=True)
filtered_test["digit"].replace(5,float(1),inplace=True)
filtered_test = filtered_test.reset_index(drop=True)

/Users/Jens/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [24]:
possible_C = [0.01, 1, 100, 10**4, 10**6]
E_ins = np.zeros(len(possible_C))
E_outs = np.zeros(len(possible_C))

for i,C in enumerate(possible_C):
    train_x = filtered[["intensity", "symmetry"]].values.tolist()
    train_y = filtered["digit"].values.tolist()
    
    test_x = filtered_test[["intensity", "symmetry"]].values.tolist()
    test_y = filtered_test["digit"].values.tolist()
    
    m = svm_train(train_y, train_x, '-q -t 2 -d 2 -c {} -g 1'.format(C))
    train_label, train_acc, train_val = svm_predict(train_y, train_x, m, "-q")
    test_label, test_acc, test_val = svm_predict(test_y, test_x, m, "-q")
    
    E_ins[i] = (100-train_acc[0])/100.
    E_outs[i] = (100-test_acc[0])/100.

## Question 9

In [25]:
possible_C[np.argmin(E_ins)]

1000000

## Question 10

In [26]:
possible_C[np.argmin(E_outs)]

100